In [ ]:
# %% [markdown]
# # Image Classification: HOG vs LBP with SVM
# 
# **Complete Analysis Notebook**
# 
# This notebook performs a comparative analysis of HOG (Histogram of Oriented Gradients) and LBP (Local Binary Patterns) feature extractors for image classification using Support Vector Machines (SVM).
# 
# ## Dataset
# - 5 classes: City, Face, Green, Office, Sea
# - Training: 695 images (~140 per class)
# - Testing: 150 images (30 per class)
# 
# ## Methodology
# 1. Load and preprocess images
# 2. Extract HOG and LBP features
# 3. Train SVM classifiers
# 4. Evaluate and compare performance
# 
# *Note: This notebook imports the modular code from the `src/` directory.*

# %% [markdown]
# ## 1. Setup and Imports

# %%
import sys
import os

# Add the src directory to Python path
sys.path.append('../src')

# Import the modular components
from config import Config
from data_loader import DataLoader
from feature_extractors import FeatureExtractor
from classifiers import ModelTrainer
from visualizer import Visualizer

# Standard imports for data manipulation and visualization
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All imports completed successfully!")

# %% [markdown]
# ## 2. Initialize Configuration and Components

# %%
# Initialize configuration
config = Config()

# Update dataset path for your local environment
config.DATASET_PATH = 'data/'  # Change this to your actual dataset path

# Initialize the components
data_loader = DataLoader(config)
feature_extractor = FeatureExtractor(config)

print("🔄 Components initialized:")
print(f"   - Dataset path: {config.DATASET_PATH}")
print(f"   - Image size: {config.IMG_SIZE}")
print(f"   - Classes: {data_loader.classes}")

# %% [markdown]
# ## 3. Load and Explore Dataset

# %%
print("📂 Loading dataset...")
X_train, X_test, y_train, y_test = data_loader.load_dataset()

# Display dataset information
print("\n📊 Dataset Summary:")
print(f"   Training images: {X_train.shape[0]}")
print(f"   Test images: {X_test.shape[0]}")
print(f"   Image shape: {X_train[0].shape}")
print(f"   Number of classes: {len(data_loader.classes)}")

# Check class distribution
train_counts = np.bincount(y_train)
test_counts = np.bincount(y_test)

print("\n🎯 Class Distribution:")
for i, class_name in enumerate(data_loader.classes):
    print(f"   {class_name}: Train={train_counts[i]}, Test={test_counts[i]}")

# %% [markdown]
# ## 4. Visualize Sample Images

# %%
# Initialize visualizer
visualizer = Visualizer(data_loader)

# Plot sample images from each class
print("🖼️ Displaying sample images from each class...")
visualizer.plot_sample_images(X_train, y_train, num_samples=3)

# %% [markdown]
# ## 5. Feature Extraction

# %%
print("🔍 Extracting HOG features...")
X_train_hog, hog_train_vis = feature_extractor.extract_hog_features(X_train)
X_test_hog, hog_test_vis = feature_extractor.extract_hog_features(X_test)

print("\n🔍 Extracting LBP features...")
X_train_lbp, lbp_train_vis = feature_extractor.extract_lbp_features(X_train)
X_test_lbp, lbp_test_vis = feature_extractor.extract_lbp_features(X_test)

print("\n📊 Feature Dimensions:")
print(f"   HOG features - Train: {X_train_hog.shape}, Test: {X_test_hog.shape}")
print(f"   LBP features - Train: {X_train_lbp.shape}, Test: {X_test_lbp.shape}")

# %% [markdown]
# ## 6. Visualize Extracted Features

# %%
print("🎨 Visualizing HOG and LBP features...")
visualizer.visualize_features(X_train, hog_train_vis, lbp_train_vis, num_examples=3)

# %% [markdown]
# ## 7. Model Training - HOG + SVM

# %%
# Initialize model trainer
trainer = ModelTrainer(config, data_loader)

print("🤖 Training HOG + SVM model...")
hog_results = trainer.train_svm_model(X_train_hog, X_test_hog, y_train, y_test, "HOG")

# Display confusion matrix
print("\n📈 HOG + SVM Confusion Matrix:")
visualizer.plot_confusion_matrix(hog_results['confusion_matrix'], "HOG + SVM")

# Save the model
os.makedirs('../results/models', exist_ok=True)
joblib.dump(hog_results['model'], '../results/models/hog_svm_model.pkl')
print("💾 HOG model saved to: ../results/models/hog_svm_model.pkl")

# %% [markdown]
# ## 8. Model Training - LBP + SVM

# %%
print("🤖 Training LBP + SVM model...")
lbp_results = trainer.train_svm_model(X_train_lbp, X_test_lbp, y_train, y_test, "LBP")

# Display confusion matrix
print("\n📈 LBP + SVM Confusion Matrix:")
visualizer.plot_confusion_matrix(lbp_results['confusion_matrix'], "LBP + SVM")

# Save the model
joblib.dump(lbp_results['model'], '../results/models/lbp_svm_model.pkl')
print("💾 LBP model saved to: ../results/models/lbp_svm_model.pkl")

# %% [markdown]
# ## 9. Performance Comparison

# %%
print("📊 Comparing HOG vs LBP performance...")
df_comparison = visualizer.compare_results(hog_results, lbp_results)

# Save comparison results
os.makedirs('../results', exist_ok=True)
results = {
    'hog_results': hog_results,
    'lbp_results': lbp_results,
    'comparison': df_comparison.to_dict(),
    'config': {
        'hog_orientations': config.HOG_ORIENTATIONS,
        'hog_pixels_per_cell': config.HOG_PIXELS_PER_CELL,
        'hog_cells_per_block': config.HOG_CELLS_PER_BLOCK,
        'lbp_radius': config.LBP_RADIUS,
        'lbp_n_points': config.LBP_N_POINTS,
        'svm_kernel': config.SVM_KERNEL,
        'svm_c': config.SVM_C
    }
}

joblib.dump(results, '../results/classification_results.pkl')
print("💾 Complete results saved to: ../results/classification_results.pkl")

# %% [markdown]
# ## 10. Additional Analysis and Visualizations

# %%
# Create a detailed comparison table
print("📋 Detailed Performance Metrics:")

comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'HOG+SVM': [
        hog_results['accuracy'],
        hog_results['precision'],
        hog_results['recall'],
        hog_results['f1']
    ],
    'LBP+SVM': [
        lbp_results['accuracy'],
        lbp_results['precision'],
        lbp_results['recall'],
        lbp_results['f1']
    ],
    'Difference': [
        hog_results['accuracy'] - lbp_results['accuracy'],
        hog_results['precision'] - lbp_results['precision'],
        hog_results['recall'] - lbp_results['recall'],
        hog_results['f1'] - lbp_results['f1']
    ]
})

print(comparison_df.to_string(index=False))

# Visualize feature distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# HOG feature distribution
axes[0].hist(X_train_hog.flatten(), bins=50, alpha=0.7, color='skyblue')
axes[0].set_title('HOG Feature Distribution')
axes[0].set_xlabel('Feature Value')
axes[0].set_ylabel('Frequency')
axes[0].grid(True, alpha=0.3)

# LBP feature distribution
axes[1].hist(X_train_lbp.flatten(), bins=50, alpha=0.7, color='lightcoral')
axes[1].set_title('LBP Feature Distribution')
axes[1].set_xlabel('Feature Value')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
print("💾 Feature distributions saved to: ../results/figures/feature_distributions.png")

# %% [markdown]
# ## 11. Test on Individual Samples

# %%
def test_single_image(image_path, model_type='hog'):
    """Test a single image with the trained model"""
    from PIL import Image
    
    # Load and preprocess image
    img = Image.open(image_path)
    img = np.array(img)
    
    if len(img.shape) == 2:  # Grayscale
        img = np.stack([img, img, img], axis=-1)
    
    # Resize to match training size
    img = cv2.resize(img, config.IMG_SIZE)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Extract features
    if model_type == 'hog':
        features, _ = feature_extractor.extract_hog_features([img])
        model = hog_results['model']
    else:
        features, _ = feature_extractor.extract_lbp_features([img])
        model = lbp_results['model']
    
    # Predict
    prediction = model.predict(features)[0]
    probability = model.predict_proba(features)[0]
    
    # Display results
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # Original image
    axes[0].imshow(img)
    axes[0].set_title('Input Image')
    axes[0].axis('off')
    
    # Prediction probabilities
    classes = data_loader.classes
    bars = axes[1].barh(classes, probability)
    axes[1].set_xlabel('Probability')
    axes[1].set_title(f'Prediction: {classes[prediction]} ({probability[prediction]:.2%})')
    axes[1].axvline(x=probability[prediction], color='red', linestyle='--', alpha=0.5)
    
    # Color the predicted class bar
    bars[prediction].set_color('red')
    
    plt.tight_layout()
    plt.show()
    
    return classes[prediction], probability

print("🔍 Single image test function ready!")
print("To test an image, run: test_single_image('path/to/image.jpg', model_type='hog')")

# %% [markdown]
# ## 12. Summary and Conclusions

# %%
print("="*70)
print("📋 PROJECT SUMMARY")
print("="*70)

print(f"\n🏆 Best Performing Method: {'HOG+SVM' if hog_results['accuracy'] > lbp_results['accuracy'] else 'LBP+SVM'}")
print(f"   Accuracy: {max(hog_results['accuracy'], lbp_results['accuracy']):.4f}")

print(f"\n📈 Key Findings:")
print(f"   1. HOG accuracy: {hog_results['accuracy']:.4f}")
print(f"   2. LBP accuracy: {lbp_results['accuracy']:.4f}")
print(f"   3. Performance difference: {abs(hog_results['accuracy'] - lbp_results['accuracy']):.4f}")

print(f"\n🔧 Parameters Used:")
print(f"   - HOG: {config.HOG_ORIENTATIONS} orientations, {config.HOG_PIXELS_PER_CELL} pixels/cell")
print(f"   - LBP: Radius={config.LBP_RADIUS}, Points={config.LBP_N_POINTS}")
print(f"   - SVM: Kernel={config.SVM_KERNEL}, C={config.SVM_C}")

print(f"\n💾 Generated Files:")
print("   - models/hog_svm_model.pkl")
print("   - models/lbp_svm_model.pkl")
print("   - classification_results.pkl")
print("   - figures/feature_visualization.png")
print("   - figures/feature_distributions.png")

print(f"\n🚀 Next Steps:")
print("   1. Experiment with different parameters")
print("   2. Try combining HOG and LBP features")
print("   3. Test on new image categories")
print("   4. Create a web interface for real-time classification")

print("\n" + "="*70)
print("✅ ANALYSIS COMPLETED SUCCESSFULLY!")
print("="*70)

# %% [markdown]
# ## 13. Save Notebook Results (Optional)

# %%
# Save the complete notebook state for reproducibility
notebook_results = {
    'X_train_shape': X_train.shape,
    'X_test_shape': X_test.shape,
    'hog_train_shape': X_train_hog.shape,
    'lbp_train_shape': X_train_lbp.shape,
    'hog_accuracy': hog_results['accuracy'],
    'lbp_accuracy': lbp_results['accuracy'],
    'config': config.__dict__
}

# Convert to DataFrame for nice display
results_df = pd.DataFrame([notebook_results])
print("📊 Final Results Summary:")
print(results_df.T)  # Transpose for better readability

# Save to CSV
results_df.to_csv('../results/notebook_summary.csv', index=False)
print("💾 Notebook summary saved to: ../results/notebook_summary.csv")

# %% [markdown]
# ---
# 
# ## How to Use This Notebook
# 
# 1. **Setup**: Update `config.DATASET_PATH` in Section 2 to your dataset location
# 2. **Run All Cells**: Use "Runtime" → "Run all" (in Colab) or "Kernel" → "Restart & Run All" (in Jupyter)
# 3. **View Results**: All outputs will be displayed inline
# 4. **Generated Files**: Check the `results/` directory for saved models and visualizations
# 
# ## Files Structure After Running:
# ```
# project/
# ├── notebooks/main_analysis.ipynb    # This notebook
# ├── src/                            # Modular code
# ├── data/                           # Dataset
# └── results/                        # Generated files
#     ├── models/                     # Trained models
#     ├── figures/                    # Visualizations
#     └── classification_results.pkl  # All results
# ```